# <font color='darkorchid'>Default Canteen</font>
<font color = 'slategray' > This notebook provides an example setup for a default reservoir and simulation using the Canteen package. This is the best staring place for new users. </font>

by: John Kucharski | 22 March 2021


In [1]:
import sys
import typing

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
sys.path.insert(0, '/Users/johnkucharski/Documents/source/canteen')
import src.reservoir as reservoir
import src.simulation as simulation

## <font color='salmon'>Reservoir</font>

A reservoir in the canteen program is made up of the following items:
* a <i>capacity</i> and <i>bottom</i> volume
* a list of <i>outlets</i>
* a dictionary functional <i>maps</i> linking volumetric locations to named variables
* a <i>name</i>

A default reservoir can be created with a single line of code.

In [3]:
default_canteen = reservoir.Reservoir()

By default the reservoir <i>capacity</i> = 1, a single spillway <i>outlet</i> is constructed, no <i>maps</i> are provided and the <i>name</i> of the reservoir is 'default'. These attributes are visable using the <i>reservoir</i> object's <i>print()</i> command.

In [4]:
default_canteen.print()

'default(capacity: 1, outlets: [spill(location: 1)], mapped variables: [None])'

## <font color='salmon'>Operations</font>

<i>Reservoir</i> operations are functions applied to the reservoir during the simulation. They are defined seperately and are not part of the <i>Reservoir</i> object. 

Some basic reservoir operations functions used to identify: target reservoir volumes (such as rule curves), target releases (such as maximum flood safe outflows and hedging rules) and releases made from each <i>outlet</i> are provided as part of the operations package. However, reservoir operations for simulation models will generally be constructed from custom made scripts. These scripted functions will generally take in a storage plus inflow volume, along with other relevant operational variables (i.e., day of year, available reservoir outlets, etc.) and return an outflows for each of the reservoir outlets.

As an example standard operating proceedures for the default reservoir are defined in the code block below.

In [5]:
def standard_operating_proceedures(res: reservoir.Reservoir, volume: float, demand: float) -> typing.Dict[str, float]:
    releases = {}
    volume_released: float = 0
    target_release: float = demand if volume - demand <= res.capacity else volume - res.capacity
    for i in range(0, len(res.outlets)):
        if volume_released < target_release and res.outlets[i].location < volume:
            release = min(target_release - volume_released, volume - res.outlets[i].location, res.outlets[i].max_release)
            releases[res.outlets[i].name] = release
            volume_released += release
            volume -= release
    return releases          

## <font color='salmon'>Simulation</font>

A simulation takes in reservoir inflows, demands and other inputs, keeps track of reservoir storage and state variables mapped to storage volumes, and reports outflow information. 